In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random
# from mtranslate import translate


2024-06-25 21:20:37.926729: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
fls = ['bestflats_ex - bestflats_ex.csv.csv',
'bitkogan - bitkogan.csv.csv',
'nlevshitstelegram - nlevshitstelegram.csv.csv',
'promtyu - promtyu.csv.csv', 
'rbc_news - rbc_news.csv.csv',
'pikabu - pikabu.csv.csv',
'sale_caviar - sale_caviar.csv.csv']

In [3]:
from datetime import timedelta
import os
ful = pd.DataFrame()
for i in fls:
    print(i.split(' ')[0])
    df = pd.read_csv(i)
    df['time'] = pd.to_datetime(df['time'])
    filename = []
    #os.listdir('/work/' + i.split(' ')[0])
    for j in range(len(df)):
        if str(df.time.iloc[j]) + '.jpg' in os.listdir(i.split(' ')[0]):
            filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j]) + '.jpg')
        else:
            if str(df.time.iloc[j] + timedelta(seconds = 1)) + '.jpg' in os.listdir(i.split(' ')[0]):
                filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j] + timedelta(seconds = 1)) + '.jpg')
            elif str(df.time.iloc[j] + timedelta(seconds = -1)) + '.jpg' in os.listdir(i.split(' ')[0]):
                
                filename.append(i.split(' ')[0]+ '/' + str(df.time.iloc[j] + timedelta(seconds = -1)) + '.jpg')
            else:
                filename.append(None)
    df['filename'] = filename
    ful = pd.concat([ful, df])

bestflats_ex
bitkogan
nlevshitstelegram
promtyu
rbc_news
pikabu
sale_caviar


In [4]:
ful = ful.dropna(subset = ['message', 'ad'])

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Conv2D, MaxPooling2D, Dropout,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [6]:
df = pd.read_csv('text.csv')

In [7]:
import random
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
import pickle

oov_tok = "<OOV>"
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(df.text)
max_len_text = 1000 
text_sequences = tokenizer.texts_to_sequences(ful['message'])
text_data = pad_sequences(text_sequences, maxlen=max_len_text)
image_data = []
for filename in ful['filename']:
    if filename is not None:
        # Load and preprocess image
        image_path = os.path.join(filename)
        img = load_img(image_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array /= 255.0  # Normalize pixel values to [0, 1]
        image_data.append(img_array)
    else:
        # If no image provided, use zeros as placeholder
        image_data.append(np.zeros((224, 224, 3)))   # Assuming image size is 224x224 and 3 channels
image_data = np.array(image_data)


In [10]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"


inputs_text = Input(shape=(max_len_text,))
inputs_image = Input(shape=(224, 224, 3))

embedding_dim = 8  
text_model = Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs_text)
text_model = GlobalAveragePooling1D()(text_model)
text_model = Dense(200, activation='relu')(text_model)
text_model = Dense(100, activation='relu')(text_model)
text_model = Dense(50, activation='relu')(text_model)
text_model = Dense(24, activation='relu')(text_model)
text_model = Dense(12, activation='relu')(text_model)

image_model = Conv2D(32, (3, 3), activation='relu')(inputs_image)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(64, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Conv2D(128, (3, 3), activation='relu')(image_model)
image_model = MaxPooling2D((2, 2))(image_model)
image_model = Flatten()(image_model)
image_model = Dense(256, activation='relu')(image_model)
image_model = Dropout(0.5)(image_model)

concatenated = Concatenate()([text_model, image_model])
output = Dense(1, activation='sigmoid')(concatenated)
    
model = Model(inputs=[inputs_text, inputs_image], outputs=output)
Adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['accuracy'])
model.load_weights('mm_weights.weights.h5')

X_text_train, X_text_test, X_image_train, X_image_test, y_train, y_test = train_test_split(
    text_data, image_data, ful['ad'], test_size=0.2, random_state=42)


model.fit([X_text_train, X_image_train], y_train, batch_size=8, epochs=5, validation_split=0.2)

loss, accuracy = model.evaluate([X_text_test, X_image_test], y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/5
310/310 [==============================] - 88s 276ms/step - loss: 0.6176 - auc: 0.5625 - val_loss: 0.3459 - val_auc: 0.5324
Epoch 2/5
310/310 [==============================] - 81s 262ms/step - loss: 0.2974 - auc: 0.5745 - val_loss: 0.2698 - val_auc: 0.6116
Epoch 3/5
310/310 [==============================] - 83s 268ms/step - loss: 0.2405 - auc: 0.6535 - val_loss: 0.2555 - val_auc: 0.7028
Epoch 4/5
310/310 [==============================] - 82s 265ms/step - loss: 0.2144 - auc: 0.7165 - val_loss: 0.2439 - val_auc: 0.7203
Epoch 5/5
25/25 [==============================] - 5s 183ms/step - loss: 0.2615 - auc: 0.5447
Test Loss: 0.26154521107673645
Test Accuracy: 0.5447075366973877


In [ ]:
# Epoch 1/10
# 78/78 [==============================] - 66s 817ms/step - loss: 0.8458 - accuracy: 0.8629 - val_loss: 0.4481 - val_accuracy: 0.8918
# Epoch 2/10
# 78/78 [==============================] - 64s 825ms/step - loss: 0.3961 - accuracy: 0.9021 - val_loss: 0.3438 - val_accuracy: 0.9079
# Epoch 3/10
# 78/78 [==============================] - 62s 797ms/step - loss: 0.3045 - accuracy: 0.9236 - val_loss: 0.2977 - val_accuracy: 0.9111
# Epoch 4/10
# 78/78 [==============================] - 64s 822ms/step - loss: 0.2778 - accuracy: 0.9248 - val_loss: 0.2824 - val_accuracy: 0.9273
# Epoch 5/10
# 78/78 [==============================] - 62s 798ms/step - loss: 0.2471 - accuracy: 0.9373 - val_loss: 0.2718 - val_accuracy: 0.9289
# Epoch 6/10
# 78/78 [==============================] - 62s 792ms/step - loss: 0.2289 - accuracy: 0.9422 - val_loss: 0.2704 - val_accuracy: 0.9257
# Epoch 7/10
# 78/78 [==============================] - 62s 790ms/step - loss: 0.2082 - accuracy: 0.9430 - val_loss: 0.2618 - val_accuracy: 0.9289
# Epoch 8/10
# 78/78 [==============================] - 62s 802ms/step - loss: 0.2063 - accuracy: 0.9422 - val_loss: 0.2676 - val_accuracy: 0.9257
# Epoch 9/10
# 78/78 [==============================] - 61s 786ms/step - loss: 0.1955 - accuracy: 0.9450 - val_loss: 0.2560 - val_accuracy: 0.9289
# Epoch 10/10
# 78/78 [==============================] - 61s 784ms/step - loss: 0.1896 - accuracy: 0.9442 - val_loss: 0.2489 - val_accuracy: 0.9289
# 25/25 [==============================] - 5s 184ms/step - loss: 0.2726 - accuracy: 0.9341
# Test Loss: 0.2726414203643799
# Test Accuracy: 0.934108555316925

In [ ]:
# Epoch 1/10
# 155/155 [==============================] - 74s 459ms/step - loss: 0.7457 - accuracy: 0.8658 - val_loss: 0.3905 - val_accuracy: 0.8966
# Epoch 2/10
# 155/155 [==============================] - 68s 442ms/step - loss: 0.3374 - accuracy: 0.9118 - val_loss: 0.2915 - val_accuracy: 0.9144
# Epoch 3/10
# 155/155 [==============================] - 69s 448ms/step - loss: 0.2616 - accuracy: 0.9325 - val_loss: 0.2641 - val_accuracy: 0.9257
# Epoch 4/10
# 155/155 [==============================] - 68s 437ms/step - loss: 0.2362 - accuracy: 0.9361 - val_loss: 0.2528 - val_accuracy: 0.9289
# Epoch 5/10
# 155/155 [==============================] - 68s 441ms/step - loss: 0.2179 - accuracy: 0.9402 - val_loss: 0.2507 - val_accuracy: 0.9289
# Epoch 6/10
# 155/155 [==============================] - 68s 437ms/step - loss: 0.2046 - accuracy: 0.9454 - val_loss: 0.2452 - val_accuracy: 0.9289
# Epoch 7/10
# 155/155 [==============================] - 68s 440ms/step - loss: 0.1956 - accuracy: 0.9474 - val_loss: 0.2461 - val_accuracy: 0.9289
# Epoch 8/10
# 155/155 [==============================] - 68s 440ms/step - loss: 0.1795 - accuracy: 0.9499 - val_loss: 0.2793 - val_accuracy: 0.9273
# Epoch 9/10
# 155/155 [==============================] - 68s 441ms/step - loss: 0.1739 - accuracy: 0.9523 - val_loss: 0.2620 - val_accuracy: 0.9289
# Epoch 10/10
# 155/155 [==============================] - 68s 439ms/step - loss: 0.1665 - accuracy: 0.9523 - val_loss: 0.2605 - val_accuracy: 0.9289
# 25/25 [==============================] - 4s 178ms/step - loss: 0.2883 - accuracy: 0.9328
# Test Loss: 0.28826966881752014
# Test Accuracy: 0.9328165650367737
